<a href="https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/namd_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Load google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Download and prepare the charmm-gui files
import pandas as pd
import os
import shutil

# Load the CSV file
df = pd.read_csv('/content/test.csv')

# Get the jobids from the first column
jobids = df.iloc[:, 0].tolist()

# Define a list to store the names of the extracted folders
extracted_folders = []

# Loop over each jobid
for jobid in jobids:
    # Download the CHARMM-GUI files
    os.system(f"curl 'https://www.charmm-gui.org/?doc=input/download&jobid={jobid}' \
    -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
    -H 'Accept-Language: en-US,en;q=0.9' \
    -H 'Connection: keep-alive' \
    -H 'Cookie: amp_6e403e=nuofvFynNKI6uifCRx6ccn...1h3u0uikt.1h3u0uimq.0.bf.bf; PHPSESSID=p8mrpcf8vms6hin3vlfnv9kn5g; visit=ok; SL_G_WPT_TO=en; SL_GWPT_Show_Hide_tmp=1; SL_wptGlobTipTmp=1; phpbb3_8fjan_k=; phpbb3_8fjan_u=30723; phpbb3_8fjan_sid=44add4206b27f77f2636d88ff3e8a140' \
    -H 'DNT: 1' \
    -H 'Referer: https://www.charmm-gui.org/?doc=input/solution&step=4&project=solution&jobid={jobid}' \
    -H 'Sec-Fetch-Dest: document' \
    -H 'Sec-Fetch-Mode: navigate' \
    -H 'Sec-Fetch-Site: same-origin' \
    -H 'Upgrade-Insecure-Requests: 1' \
    -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0' \
    -H 'sec-ch-ua: \"Microsoft Edge\";v=\"123\", \"Not:A-Brand\";v=\"8\", \"Chromium\";v=\"123\"' \
    -H 'sec-ch-ua-mobile: ?0' \
    -H 'sec-ch-ua-platform: \"Windows\"' \
    --compressed \
    --output charmm-gui.tgz")
    
    # Extract the CHARMM-GUI files
    os.system("tar -zxf charmm-gui.tgz")

    # Define the name of the extracted folder
    extracted_folder = f"charmm-gui-{jobid}"

    # Add the name of the extracted folder to the list
    extracted_folders.append(extracted_folder)

    # Remove the downloaded .tgz file
    os.remove("charmm-gui.tgz")

# Move the extracted folders to /content/drive/MyDrive/namd after all the downloads and extractions are done
for folder in extracted_folders:
    shutil.move(folder, f"/content/drive/MyDrive/namd/{folder}")

In [ ]:
#@title Install NAMD
# Download the NAMD
!wget https://www.ks.uiuc.edu/Research/namd/3.0b6/download/120834/NAMD_3.0b6_Linux-x86_64-multicore-CUDA.tar.gz
#Extract NAMD
!tar -zxf /content/NAMD_3.0b6_Linux-x86_64-multicore-CUDA.tar.gz

In [ ]:
#@title Equilibration run

# Get the list of all directories in /MyDrive/namd
folders = os.listdir('/content/drive/MyDrive/namd')

# Loop over each folder
for folder in folders:
    # Construct the file path
    file_path = f"/content/drive/MyDrive/namd/{folder}/namd"
    
    # Change directory to the file path
    os.chdir(file_path)
    
    # Run the NAMD simulation
    !/content/NAMD_3.0b4_Linux-x86_64-multicore-CUDA.tar.gz +auto-provision +idlepoll +devices 0 ./step4_equilibration.inp > ./step4_equilibration.log &


In [ ]:
#@title Simulation run
import os
import fileinput

# Get the list of all directories in /MyDrive/namd
folders = os.listdir('/content/drive/MyDrive/namd')

# Loop over each folder
for folder in folders:
    # Construct the file path
    file_path = f"/content/drive/MyDrive/namd/{folder}/namd"

    # Change directory to the file path
    os.chdir(file_path)

    # Open the step4_equilibration.inp file
    with fileinput.FileInput("step4_equilibration.inp", inplace=True) as file:
        # Loop through each line in the file
        for line in file:
            # If the line contains "run", replace it with the new value
            if "run" in line:
                print(f"run                500000")
            else:
                print(line.rstrip())

    # Run the NAMD simulation
    !./NAMD_3.0b4_Linux-x86_64-multicore-CUDA.tar.gz +auto-provision +idlepoll +devices 0 ./step4_equilibration.inp > ./step4_equilibration.log &

In [ ]:
#@title Copy file to GDrive
#ls /content/drive/MyDrive/namd/B1/B1/namd
!cp *.log *.coor *.vel *.xsc *.xst *.dcd /content/drive/MyDrive

To combine multiples .dcd files, see this notebook:https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/CatDCD.ipynb

References:

1. https://www.youtube.com/watch?v=NHb5s9zDZP8
2. James C. Phillips, David J. Hardy, Julio D. C. Maia, John E. Stone, João V. Ribeiro, Rafael C. Bernardi, Ronak Buch, Giacomo Fiorin, Jérôme Hénin, Wei Jiang, Ryan McGreevy, Marcelo C. R. Melo, Brian Radak, Robert D. Skeel, Abhishek Singharoy, Yi Wang, Benoît Roux, Aleksei Aksimentiev, Zaida Luthey-Schulten, Laxmikant V. Kalé, Klaus Schulten, Christophe Chipot, and Emad Tajkhorshid. Scalable molecular dynamics on CPU and GPU architectures with NAMD. Journal of Chemical Physics, 153:044130, 2020. (PMC: PMC7395834)

